# Amazon Personalize

## How to Use the Notebook

Code is broken up into cells like the one below. There's a triangular `Run` button at the top of this page you can click to execute each cell and move onto the next, or you can press `Shift` + `Enter` while in the cell to execute it and move onto the next one.

As a cell is executing you'll notice a line to the side showcase an `*` while the cell is running or it will update to a number to indicate the last cell that completed executing after it has finished exectuting all the code within a cell.


Simply follow the instructions below and execute the cells to get started with Amazon Personalize.

## 0. Imports 

Python ships with a broad collection of libraries and we need to import those as well as the ones installed to help us like [boto3](https://aws.amazon.com/sdk-for-python/) (AWS SDK for python) and [Pandas](https://pandas.pydata.org/)/[Numpy](https://numpy.org/) which are core data science tools.

In [13]:
!pip install -U sagemaker

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 660.4/660.4 kB 14.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 45.2 MB/s eta 0:00:0000:0100:01
  Created wheel for sagemaker: filename=sagemaker-2.128.0-py2.py3-none-any.whl size=896997 sha256=1d19f096ffeddeb0b80b9c0ba4a42d8f05c1e884c87a49ca2cb35157002e7ba1
  Stored in directory: /home/ec2-user/.cache/pip/wheels/e1/e9/4d/1a058f1fcdae844c8b70f096e8436a08d147801a6dd1216a40
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.24.19
    Uninstalling botocore-1.24.19:
      Successfully uninstalled botocore-1.24.19
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.127.0
    Uninstalling sagemaker-2.127.0:
      Successfully uninstalled sagemaker-2.127.0
ERROR: pip's dependency resolver does not currently ta

In [1]:
# Imports
import boto3
import sagemaker
import json
import numpy as np
import pandas as pd
import time
import random
# !conda install -y -c conda-forge unzip

Next you will want to validate that your environment can communicate successfully with Amazon Personalize, the lines below do just that.

## 1. Define Boto3, Policy and Role

In [24]:
date = '20231012'

In [25]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

In [26]:
bucket = sagemaker.Session().default_bucket()

### 1-1. Attach Policy to S3 Bucket

Amazon Personalize needs to be able to read the content of your S3 bucket that you created earlier. The lines below will do that.

In [27]:
s3 = boto3.client("s3")

policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:GetObject",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket),
                "arn:aws:s3:::{}/*".format(bucket)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': 'G6MEXBSEH48X521R',
  'HostId': 'GyfWzEYGeMqQ7/HlAVqXtrMKaWWHk3cmD517L2TqzCuuwqcz9jV9OYZwP6OXTrFlUeLvlrMBzuE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'GyfWzEYGeMqQ7/HlAVqXtrMKaWWHk3cmD517L2TqzCuuwqcz9jV9OYZwP6OXTrFlUeLvlrMBzuE=',
   'x-amz-request-id': 'G6MEXBSEH48X521R',
   'date': 'Wed, 11 Jan 2023 07:37:08 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### 1-2. Create Personalize Role

Also Amazon Personalize needs the ability to assume Roles in AWS in order to have the permissions to execute certain tasks, the lines below grant that.

In [28]:
iam = boto3.client("iam")

role_name = f"PersonalizeRoleDemo-{date}"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(10) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::426989617652:role/PersonalizeRoleDemo-20231012


## 2. Create Sample Dataset

### 2-1. Interactions Dataset

In [29]:
from datetime import datetime, timedelta
import time

TIMESTAMP = time.mktime((datetime.now()-timedelta(days=100)).timetuple())
TIMESTAMP = int(TIMESTAMP)
TIMESTAMP

1664782637

In [30]:
user_id = [i+1 for i in range(100)]
item_id = [i+1000 for i in range(1000)]
item_type = ['AAA','BBB']
event_val = [i+1 for i in range(3)]
event_type = ['purchased','checked']
print(f"USER_ID : {user_id[:10]} \nITEM_ID : {item_id[:10]} \nITEM_TYPE : {item_type} \nEVENT_VALUE : {event_val} \nEVENT_TYPE : {event_type}")

USER_ID : [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
ITEM_ID : [1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009] 
ITEM_TYPE : ['AAA', 'BBB'] 
EVENT_VALUE : [1, 2, 3] 
EVENT_TYPE : ['purchased', 'checked']


In [31]:
item_mapping={}
for item in item_id:
    item_mapping[item] = random.choice(item_type)

In [32]:
interaction_data = []
for i in range(100000):
    event_type_tmp = random.choice(event_type)
    if event_type_tmp == 'checked':
        event_val_tmp = random.choice(event_val)
    else:
        event_val_tmp = None
    select_item_id = random.choice(item_id)
    select_item_type = item_mapping[select_item_id]
    interaction_data.append([random.choice(user_id),select_item_id, 
                 TIMESTAMP+random.choice(range(10000,4320108)), select_item_type,
                 event_val_tmp, event_type_tmp])

In [33]:
interaction_pd_data=pd.DataFrame(interaction_data, columns=['USER_ID', 'ITEM_ID', 'TIMESTAMP', 'ITEM_TYPE', 'EVENT_VALUE', 'EVENT_TYPE'])
interaction_pd_data.head()

,USER_ID,ITEM_ID,TIMESTAMP,ITEM_TYPE,EVENT_VALUE,EVENT_TYPE
0,84,1047,1666813614,AAA,3.0,checked
1,98,1607,1668926061,AAA,2.0,checked
2,10,1141,1667494589,BBB,2.0,checked
3,100,1839,1666536060,AAA,NaN,purchased
4,65,1383,1665255944,BBB,2.0,checked


In [34]:
sample_interaction_filename = f"interaction_sample_{date}.csv"
interaction_pd_data.to_csv(sample_interaction_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(sample_interaction_filename).upload_file(sample_interaction_filename)

### 2-2. Users dataset

In [35]:
user_meta_1 = []

for i in range(3):
    user_meta_1.append(f'USER_GROUP_{i}')

user_meta_2 = [i for i in range(100,103)]

In [36]:
user_data = []
for i in user_id:
    user_data.append([i,random.choice(user_meta_1),random.choice(user_meta_2)])

In [37]:
user_pd_data=pd.DataFrame(user_data, columns=['USER_ID', 'USER_META1', 'USER_META2'])
user_pd_data.head()

,USER_ID,USER_META1,USER_META2
0,1,USER_GROUP_1,100
1,2,USER_GROUP_1,100
2,3,USER_GROUP_0,100
3,4,USER_GROUP_2,102
4,5,USER_GROUP_2,102


In [38]:
sample_user_filename = f"users_sample_{date}.csv"
user_pd_data.to_csv(sample_user_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(sample_user_filename).upload_file(sample_user_filename)

### 2-3. Items dataset

In [39]:
text = "food is any substance consumed to provide nutritional support for an organism. Food is usually of plant, animal, or fungal origin, and contains essential nutrients, such as carbohydrates, fats, proteins, vitamins, or minerals. The substance is ingested by an organism and assimilated by the organism's cells to provide energy, maintain life, or stimulate growth. Different species of animals have different feeding behaviours that satisfy the needs of their unique metabolisms, often evolved to fill a specific ecological niche within specific geographical contexts. Omnivorous humans are highly adaptable and have adapted to obtain food in many different ecosystems. Historically, humans secured food through two main methods: hunting and gathering and agriculture. As agricultural technologies increased, humans settled into agriculture lifestyles with diets shaped by the agriculture opportunities in their geography. Geographic and cultural differences has led to creation of numerous cuisines and culinary arts, including a wide array of ingredients, herbs, spices, techniques, and dishes. As cultures have mixed through forces like international trade and globalization, ingredients have become more widely available beyond their geographic and cultural origins, creating a cosmopolitan exchange of different food traditions and practices. Today, the majority of the food energy required by the ever-increasing population of the world is supplied by the industrial food industry, which produces food with intensive agriculture and distributes it through complex food processing and food distribution systems. This system of conventional agriculture relies heavily on fossil fuels, which means that the food and agricultural system is one of the major contributors to climate change, accountable for as much as 37% of total greenhouse gas emissions. Addressing the carbon intensity of the food system and food waste are important mitigation measures in the global response to climate change. The food system has significant impacts on a wide range of other social and political issues including: sustainability, biological diversity, economics, population growth, water supply, and access to food. The right to food is a human right derived from the International Covenant on Economic, Social and Cultural Rights, recognizing the right to an adequate standard of living, including adequate food, as well as the fundamental right to be free from hunger. Because of these fundamental rights, food security is often a priority international policy activity; for example Sustainable Development Goal 2 Zero hunger is meant to eliminate hunger by 2030. Food safety and food security are monitored by international agencies like the International Association for Food Protection, World Resources Institute, World Food Programme, Food and Agriculture Organization, and International Food Information Council, and are often subject to national regulation by institutions, like the Food and Drug Administration in the United States. Food is any substance consumed to provide nutritional support and energy to an organism. It can be raw, processed or formulated and is consumed orally by animals for growth, health or pleasure. Food is mainly composed of water, lipids, proteins and carbohydrates. Minerals and organic substances vitamins can also be found in food. Plants, algae and some microorganisms use photosynthesis to make their own food molecules. Water is found in many foods and has been defined as a food by itself. Water and fiber have low energy densities, or calories, while fat is the most energy dense component. Some inorganic (non-food) elements are also essential for plant and animal functioning. Human food can be classified in various ways, either by related content or by how the food is processed. The number and composition of food groups can vary. Most systems include four basic groups that describe their origin and relative nutritional function: Vegetables and Fruit, Cereals and Bread, Dairy, and Meat. Studies that look into diet quality often group food into whole grains, cereals, refined grains/cereals, vegetables, fruits, nuts, legumes, eggs, dairy products, fish, red meat, processed meat, and sugar-sweetened beverages. The Food and Agriculture Organization and World Health Organization use a system with nineteen food classifications: cereals, roots, pulses and nuts, milk, eggs, fish and shellfish, meat, insects, vegetables, fruits, fats and oils, sweets and sugars, spices and condiments, beverages, foods for nutritional uses, food additives, composite dishes and savoury snacks.Plants as a food source are often divided into seeds, fruits, vegetables, legumes, grains and nuts. Where plants fall within these categories can vary with botanically described fruits such as the tomato, squash, pepper and eggplant or seeds like peas commonly considered vegetables. Food is a fruit if the part eaten is derived from the reproductive tissue, so seeds, nuts and grains are technically fruit. From a culinary perspective fruits are generally considered the remains of botanically described fruits after grains, nuts, seeds and fruits used as vegetables are removed. Grains can be defined as seeds that humans eat or harvest, with cereal grains oats, wheat, rice, corn, barley, rye, sorghum and millet belonging to the Poaceae grass family and pulses coming from the Fabaceae legume family. Whole grains are foods that contain all the elements of the original seed bran, germ, and endosperm. Nuts are dry fruits distinguishable by their woody shell."

In [40]:
item_meta_1 = []

for i in range(3):
    item_meta_1.append(f'ITEM_GROUP_{i+1}')

item_meta_2 = []

for i in range(3):
    item_meta_2.append(f'ITEM_GROUP_10000{i+1}')

candidate_desc = text.split(' ')

In [41]:
item_data = []
for i in item_id:
    description = []
    desc_text = ''
    for word in random.sample(candidate_desc, random.choice(range(1,5))):
        desc_text += word + ' '
#     description.append(desc_text)
    select_item_meta_1 = random.choice(item_meta_1)
    select_item_meta_2 = random.choice(item_meta_2)
    
    item_data.append([i,select_item_meta_1,select_item_meta_2,
                      TIMESTAMP+random.choice(range(10000,4320108)), 
                      desc_text])

In [42]:
item_pd_data=pd.DataFrame(item_data, columns=['ITEM_ID','ITEM_META1','ITEM_META2','CREATION_TIMESTAMP','DESCRIPTION'])
item_pd_data.head()

,ITEM_ID,ITEM_META1,ITEM_META2,CREATION_TIMESTAMP,DESCRIPTION
0,1000,ITEM_GROUP_3,ITEM_GROUP_100003,1665638038,and
1,1001,ITEM_GROUP_2,ITEM_GROUP_100002,1665322759,provide World
2,1002,ITEM_GROUP_1,ITEM_GROUP_100002,1665609197,by
3,1003,ITEM_GROUP_1,ITEM_GROUP_100003,1664824912,"legumes,"
4,1004,ITEM_GROUP_2,ITEM_GROUP_100001,1667102038,culinary


In [43]:
sample_item_filename = f"items_sample_{date}.csv"
item_pd_data.to_csv(sample_item_filename, index=False)
boto3.Session().resource('s3').Bucket(bucket).Object(sample_item_filename).upload_file(sample_item_filename)

## 3. Create Schema

A core component of how Personalize understands your data comes from the Schema that is defined below. This configuration tells the service how to digest the data provided via your CSV file. Note the columns and types align to what was in the file you created above.

### 3-1. Interactions datasets schema

In [44]:
interactions_schema_name = f'interactions-samples-{date}'
try:
    personalize.delete_schema(schemaArn=f'arn:aws:personalize:us-west-2:687314952804:schema/{interactions_schema_name}')
except:
    print("The schema doesn't exist")

The schema doesn't exist


In [45]:
interactions_schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        },
        {
            "name": "ITEM_TYPE",
            "type": "string",
            "categorical": True
        },
        {
            "name": "EVENT_VALUE",
            "type": [
             "float",
             "null"
          ]
        },
        {
            "name": "EVENT_TYPE",
            "type": "string"
        }
    ],
    "version": "1.0"
}

interactions_create_schema_response = personalize.create_schema(
    name = interactions_schema_name,
    schema = json.dumps(interactions_schema)
)

interactions_schema_arn = interactions_create_schema_response['schemaArn']
print(json.dumps(interactions_create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:426989617652:schema/interactions-samples-20231012",
  "ResponseMetadata": {
    "RequestId": "76fd24e6-6e04-4d04-a618-3e3e73b26caa",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 07:37:20 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "95",
      "connection": "keep-alive",
      "x-amzn-requestid": "76fd24e6-6e04-4d04-a618-3e3e73b26caa"
    },
    "RetryAttempts": 0
  }
}


### 3-2. User datasets schema

In [46]:
users_schema_name = f'user-samples-{date}'
try:
    personalize.delete_schema(schemaArn=f'arn:aws:personalize:us-west-2:687314952804:schema/{users_schema_name}')
except:
    print("The schema doesn't exist")


The schema doesn't exist


In [47]:
users_schema = {
    "type": "record",
    "name": "Users",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "USER_META1",
            "type": "string",
            "categorical": True
        },
        {
            "name": "USER_META2",
            "type": "int"
        }
    ],
    "version": "1.0"
}

users_create_schema_response = personalize.create_schema(
    name = users_schema_name,
    schema = json.dumps(users_schema)
)

users_schema_arn = users_create_schema_response['schemaArn']
print(json.dumps(users_create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:426989617652:schema/user-samples-20231012",
  "ResponseMetadata": {
    "RequestId": "dd31ba0e-c1c2-45b2-bdfa-931ef3aaaee2",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 07:37:21 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "87",
      "connection": "keep-alive",
      "x-amzn-requestid": "dd31ba0e-c1c2-45b2-bdfa-931ef3aaaee2"
    },
    "RetryAttempts": 0
  }
}


### 3-3. Items datasets schema

In [48]:
items_schema_name = f'items-samples-{date}'
try:
    personalize.delete_schema(schemaArn=f'arn:aws:personalize:us-west-2:687314952804:schema/{items_schema_name}')
except:
    print("The schema doesn't exist")


The schema doesn't exist


In [49]:
items_schema = {
    "type": "record",
    "name": "Items",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "ITEM_META1",
            "type": [
                "null",
                "string"
              ],
              "categorical": True
        },
        {
            "name": "ITEM_META2",
            "type": [
                "null",
                "string"
              ],
              "categorical": True
        },
        {
          "name": "CREATION_TIMESTAMP",
          "type": "long"
        },
        {
          "name": "DESCRIPTION",
          "type": [
            "null",
            "string"
          ],
          "textual": True
        },
    ],
    "version": "1.0"
}

items_create_schema_response = personalize.create_schema(
    name = items_schema_name,
    schema = json.dumps(items_schema)
)

items_schema_arn = items_create_schema_response['schemaArn']
print(json.dumps(items_create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:426989617652:schema/items-samples-20231012",
  "ResponseMetadata": {
    "RequestId": "5870c55d-68f4-4595-9e9a-ee18eda361ab",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 07:37:22 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "88",
      "connection": "keep-alive",
      "x-amzn-requestid": "5870c55d-68f4-4595-9e9a-ee18eda361ab"
    },
    "RetryAttempts": 0
  }
}


## 4. Create Dataset Group

The largest grouping in Personalize is a Dataset Group, this will isolate your data, event trackers, solutions, and campaigns. Grouping things together that share a common collection of data. Feel free to alter the name below if you'd like.

### 4-1. Dataset group details

In [62]:
create_dataset_group_response = personalize.create_dataset_group(
    name = f"dataset-samples-{date}"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

ResourceAlreadyExistsException: An error occurred (ResourceAlreadyExistsException) when calling the CreateDatasetGroup operation: Another resource with Arn arn:aws:personalize:us-east-1:426989617652:dataset-group/dataset-samples-20231012 already exists.

##### Wait for Dataset Group to Have ACTIVE Status

Before we can use the Dataset Group in any items below it must be active, execute the cell below and wait for it to show active.

In [63]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(10)

DatasetGroup: ACTIVE


### 4-2. Create Datasets

After the group, the next thing to create is the actual datasets. Execute the cells below to create it.

#### 4-2-1. Interactions Datasets

In [64]:
interactions_dataset_type = "INTERACTIONS"
try:
    interactions_create_dataset_response = personalize.create_dataset(
        name = f"interactions-samples-dataset-{date}",
        datasetType = interactions_dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = interactions_schema_arn
    )

    interactions_dataset_arn = interactions_create_dataset_response['datasetArn']
    print(json.dumps(interactions_create_dataset_response, indent=2))
except Exception as e:
    print(e)

An error occurred (ResourceAlreadyExistsException) when calling the CreateDataset operation: Another dataset of type: INTERACTIONS already exists in the provided dataset group


#### 4-2-2. Users Datasets

In [65]:
users_dataset_type = "USERS"
try:
    users_create_dataset_response = personalize.create_dataset(
        name = f"users-samples-dataset-{date}",
        datasetType = users_dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = users_schema_arn
    )

    users_dataset_arn = users_create_dataset_response['datasetArn']
    print(json.dumps(users_create_dataset_response, indent=2))
except Exception as e:
    print(e)

An error occurred (ResourceAlreadyExistsException) when calling the CreateDataset operation: Another dataset of type: USERS already exists in the provided dataset group


#### 4-2-3. Items Datasets

In [66]:
items_dataset_type = "ITEMS"
try:
    items_create_dataset_response = personalize.create_dataset(
        name = f"items-samples-dataset-{date}",
        datasetType = items_dataset_type,
        datasetGroupArn = dataset_group_arn,
        schemaArn = items_schema_arn
    )

    items_dataset_arn = items_create_dataset_response['datasetArn']
    print(json.dumps(items_create_dataset_response, indent=2))
except Exception as e:
    print(e)

An error occurred (ResourceAlreadyExistsException) when calling the CreateDataset operation: Another dataset of type: ITEMS already exists in the provided dataset group


## 4-3. Dataset Import jobs

### 4-3-1. Create dataset import job for Interactions

In [67]:
time.sleep(10)
try:
    interactions_create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = f"import-sample-interactions-{date}",
        datasetArn = interactions_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket, sample_interaction_filename)
        },
        roleArn = role_arn
    )

    interactions_dataset_import_job_arn = interactions_create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(interactions_create_dataset_import_job_response, indent=2))
except Exception as e:
    print(e)

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:426989617652:dataset-import-job/import-sample-interactions-20231012",
  "ResponseMetadata": {
    "RequestId": "9926e852-1595-447e-8416-a11a3fd6d4b7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 07:59:49 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "123",
      "connection": "keep-alive",
      "x-amzn-requestid": "9926e852-1595-447e-8416-a11a3fd6d4b7"
    },
    "RetryAttempts": 0
  }
}


### 4-3-2. Create dataset import job for Users

In [68]:
time.sleep(10)
try:
    users_create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = f"import-sample-users-{date}",
        datasetArn = users_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket, sample_user_filename)
        },
        roleArn = role_arn
    )

    users_dataset_import_job_arn = users_create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(users_dataset_import_job_arn, indent=2))
except Exception as e:
    print(e)

An error occurred (ResourceAlreadyExistsException) when calling the CreateDatasetImportJob operation: Another resource with Arn arn:aws:personalize:us-east-1:426989617652:dataset-import-job/import-sample-users-20231012 already exists.


### 4-3-3. Create dataset import job for Items

In [69]:
time.sleep(10)
try:
    items_create_dataset_import_job_response = personalize.create_dataset_import_job(
        jobName = f"import-sample-items-{date}",
        datasetArn = items_dataset_arn,
        dataSource = {
            "dataLocation": "s3://{}/{}".format(bucket, sample_item_filename)
        },
        roleArn = role_arn
    )

    items_dataset_import_job_arn = items_create_dataset_import_job_response['datasetImportJobArn']
    print(json.dumps(items_dataset_import_job_arn, indent=2))
except Exception as e:
    print(e)

An error occurred (ResourceAlreadyExistsException) when calling the CreateDatasetImportJob operation: Another resource with Arn arn:aws:personalize:us-east-1:426989617652:dataset-import-job/import-sample-items-20231012 already exists.


##### Wait for Dataset Import Job to Have ACTIVE Status

It can take a while before the import job completes, please wait until you see that it is active below.

In [74]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = items_dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(10)

DatasetImportJob: ACTIVE


## 5. Use custom resources

In Amazon Personalize a trained model is called a Solution, each Solution can have many specific versions that relate to a given volume of data when the model was trained.

To begin we will list all the recipies that are supported, a recipie is an algorithm that has not been trained on your data yet. After listing you'll select one and use that to build your model.

### 5-1. Select Recipe

In [75]:
list_recipes_response = personalize.list_recipes()
list_recipes = [list_recipes_response]
while True:
    if list_recipes_response.get('nextToken'):
        list_recipes_response = personalize.list_recipes(nextToken=list_recipes_response['nextToken'])
        list_recipes.append(list_recipes_response)
    else:
        break

num = 1
for response in list_recipes:
    for i, recipes_name in enumerate(response['recipes']):
        name = recipes_name['name']
        recipeArn = recipes_name['recipeArn']

        if recipes_name.get('domain'):
            domain = recipes_name['domain']
            print(f"{num}.{name}, {recipeArn}, {domain}")
        else:
            print(f"{num}.{name}, {recipeArn}")
        num += 1

1.aws-ecomm-customers-who-viewed-x-also-viewed, arn:aws:personalize:::recipe/aws-ecomm-customers-who-viewed-x-also-viewed, ECOMMERCE
2.aws-ecomm-frequently-bought-together, arn:aws:personalize:::recipe/aws-ecomm-frequently-bought-together, ECOMMERCE
3.aws-ecomm-popular-items-by-purchases, arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-purchases, ECOMMERCE
4.aws-ecomm-popular-items-by-views, arn:aws:personalize:::recipe/aws-ecomm-popular-items-by-views, ECOMMERCE
5.aws-ecomm-recommended-for-you, arn:aws:personalize:::recipe/aws-ecomm-recommended-for-you, ECOMMERCE
6.aws-item-affinity, arn:aws:personalize:::recipe/aws-item-affinity
7.aws-item-attribute-affinity, arn:aws:personalize:::recipe/aws-item-attribute-affinity
8.aws-personalized-ranking, arn:aws:personalize:::recipe/aws-personalized-ranking
9.aws-popularity-count, arn:aws:personalize:::recipe/aws-popularity-count
10.aws-similar-items, arn:aws:personalize:::recipe/aws-similar-items
11.aws-sims, arn:aws:personalize:::recip

### 5-2. Recipe - User Personalization
The [User-Personalization](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-new-item-USER_PERSONALIZATION.html) (aws-user-personalization) recipe is optimized for all USER_PERSONALIZATION recommendation scenarios. When recommending items, it uses automatic item exploration.

With automatic exploration, Amazon Personalize automatically tests different item recommendations, learns from how users interact with these recommended items, and boosts recommendations for items that drive better engagement and conversion. This improves item discovery and engagement when you have a fast-changing catalog, or when new items, such as news articles or promotions, are more relevant to users when fresh.

You can balance how much to explore (where items with less interactions data or relevance are recommended more frequently) against how much to exploit (where recommendations are based on what we know or relevance). Amazon Personalize automatically adjusts future recommendations based on implicit user feedback.

First, select the recipe by finding the ARN in the list of recipes above.

In [76]:
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization" # aws-user-personalization selected for demo purposes

### 5-3. Create Solution

First you will create the solution with the API, then you will create a version. It will take several minutes to train the model and thus create your version of a solution. Once it gets started and you are seeing the in progress notifications it is a good time to take a break, grab a coffee, etc.

In [77]:
try:
    create_solution_response = personalize.create_solution(
        name = f"personalize-sample-user-personalization-{date}",
        datasetGroupArn = dataset_group_arn,
        recipeArn = recipe_arn
    )

    solution_arn = create_solution_response['solutionArn']
    print(json.dumps(create_solution_response, indent=2))
except Exception as e:
    print(e)
    solution_arn = create_solution_response['solutionArn']
    print(f"solution_arn : {solution_arn}")

{
  "solutionArn": "arn:aws:personalize:us-east-1:426989617652:solution/personalize-sample-user-personalization-20231012",
  "ResponseMetadata": {
    "RequestId": "4d8b5753-e87e-4e7d-a36b-a89b2468171c",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 08:04:48 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "118",
      "connection": "keep-alive",
      "x-amzn-requestid": "4d8b5753-e87e-4e7d-a36b-a89b2468171c"
    },
    "RetryAttempts": 0
  }
}


### 5-4. Create Solution Version

Personalize 재학습에서는 trainingMode='UPDATE'로 변경해서 아래 작업을 다시 진행해 볼 수 있습니다.

In [78]:
trainingMode = "FULL"
# trainingMode = "UPDATE"

In [79]:
try:
    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn,
        trainingMode=trainingMode 
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print(json.dumps(create_solution_version_response, indent=2))
except Exception as e:
    print(e)

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:426989617652:solution/personalize-sample-user-personalization-20231012/f6bb5f2b",
  "ResponseMetadata": {
    "RequestId": "52b473d7-8771-4687-898a-1785775f0f13",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 08:04:50 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "connection": "keep-alive",
      "x-amzn-requestid": "52b473d7-8771-4687-898a-1785775f0f13"
    },
    "RetryAttempts": 0
  }
}


##### Wait for Solution Version to Have ACTIVE Status

This will take approximately 40-50 minutes.

In [80]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGR

--------
### 7-2. Create new filter
시간 관계 상 뒤에서 사용할 Filter를 사전에 만드는 작업을 진행합니다.

In [83]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
region_name = boto3.Session().region_name

filter_name = f'sample-filter-not-in-check-and-purchased-{date}'
filter_arn = f"arn:aws:personalize:{region_name}:{account_id}:filter/{filter_name}"
try:
    personalize.delete_filter(
        filterArn=filter_arn
    )
except:
    pass

try:
    res_filter = personalize.create_filter(
        name=filter_name,
        datasetGroupArn=dataset_group_arn,
        filterExpression='EXCLUDE ItemID WHERE Interactions.event_type IN ("checked", "purchased")'
    )
except Exception as e:
    print(e)

##### Wait for creating a filter to Have ACTIVE Status

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_filter_response = personalize.describe_filter(
        filterArn = filter_arn
    )
    status = describe_filter_response["filter"]["status"]
    print("Filter: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(10)

Filter: CREATE PENDING
Filter: CREATE PENDING
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: CREATE IN_PROGRESS
Filter: C

--------
### 5-5. Get Metrics of Solution Version

Now that your solution and version exists, you can obtain the metrics for it to judge its performance. These metrics are not particularly good as it is a demo set of data, but with larger more complex datasets you should see improvements.

In [84]:
try:
    get_solution_metrics_response = personalize.get_solution_metrics(
        solutionVersionArn = solution_version_arn
    )
    print(json.dumps(get_solution_metrics_response, indent=2))
except Exception as e:
    print(e)

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:426989617652:solution/personalize-sample-user-personalization-20231012/f6bb5f2b",
  "metrics": {
    "coverage": 0.1169,
    "mean_reciprocal_rank_at_25": 0.0677,
    "normalized_discounted_cumulative_gain_at_10": 0.0714,
    "normalized_discounted_cumulative_gain_at_25": 0.2086,
    "normalized_discounted_cumulative_gain_at_5": 0.0714,
    "precision_at_10": 0.0143,
    "precision_at_25": 0.0286,
    "precision_at_5": 0.0286
  },
  "ResponseMetadata": {
    "RequestId": "37bed244-719b-48f6-a6e2-30cb1e67a7f1",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 09:59:44 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "432",
      "connection": "keep-alive",
      "x-amzn-requestid": "37bed244-719b-48f6-a6e2-30cb1e67a7f1"
    },
    "RetryAttempts": 0
  }
}


We recommend reading [the documentation](https://docs.aws.amazon.com/personalize/latest/dg/working-with-training-metrics.html) to understand the metrics, but we have also copied parts of the documentation below for convenience.

You need to understand the following terms regarding evaluation in Personalize:

- *Relevant recommendation* refers to a recommendation that matches a value in the testing data for the particular user.
- *Rank* refers to the position of a recommended item in the list of recommendations. Position 1 (the top of the list) is presumed to be the most relevant to the user.
- *Query* refers to the internal equivalent of a GetRecommendations call.

The metrics produced by Personalize are:

- coverage: The proportion of unique recommended items from all queries out of the total number of unique items in the training data (includes both the Items and Interactions datasets).
- mean_reciprocal_rank_at_25: The [mean of the reciprocal ranks](https://en.wikipedia.org/wiki/Mean_reciprocal_rank) of the first relevant recommendation out of the top 25 recommendations over all queries. This metric is appropriate if you're interested in the single highest ranked recommendation.
- normalized_discounted_cumulative_gain_at_K: Discounted gain assumes that recommendations lower on a list of recommendations are less relevant than higher recommendations. Therefore, each recommendation is discounted (given a lower weight) by a factor dependent on its position. To produce the [cumulative discounted gain](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) (DCG) at K, each relevant discounted recommendation in the top K recommendations is summed together. The normalized discounted cumulative gain (NDCG) is the DCG divided by the ideal DCG such that NDCG is between 0 - 1. (The ideal DCG is where the top K recommendations are sorted by relevance.) Amazon Personalize uses a weighting factor of 1/log(1 + position), where the top of the list is position 1. This metric rewards relevant items that appear near the top of the list, because the top of a list usually draws more attention.
- precision_at_K: The number of relevant recommendations out of the top K recommendations divided by K. This metric rewards precise recommendation of the relevant items.

## 6. Create and Wait for the Campaign

Now that you have a working solution version you will need to create a campaign to use it with your applications. A campaign is a hosted solution version; an endpoint which you can query for recommendations. Pricing is set by estimating throughput capacity (requests from users for personalization per second). When deploying a campaign, you set a minimum transactions per second (TPS) value (`minProvisionedTPS`). This service, like many within AWS, will automatically scale based on demand, but if latency is critical, you may want to provision ahead for larger demand. For this demo, the minimum throughput threshold is set to 1. For more information, see the [pricing](https://aws.amazon.com/personalize/pricing/) page.

As mentioned above, the user-personalization recipe used for our solution supports automatic exploration of "cold" items. You can control how much exploration is performed when creating your campaign. The `itemExplorationConfig` data type supports `explorationWeight` and `explorationItemAgeCutOff` parameters. Exploration weight determines how frequently recommendations include items with less interactions data or relevance. The closer the value is to 1.0, the more exploration. At zero, no exploration occurs and recommendations are based on current data (relevance). Exploration item age cut-off determines items to be explored based on time frame since latest interaction. Provide the maximum item age, in days since the latest interaction, to define the scope of item exploration. The larger the value, the more items are considered during exploration. For our campaign below, we'll specify an exploration weight of 0.5.

### 6-1. Create Campaign

In [85]:
try:
    if trainingMode == "FULL":
        create_campaign_response = personalize.create_campaign(
            name = f"personalize-sample-camp-{date}",
            solutionVersionArn = solution_version_arn,
            minProvisionedTPS = 1,
            campaignConfig = {
                "itemExplorationConfig": {
                    "explorationWeight": "0.5",   ### Exploration 비율을 조정해 볼 수 있는 값
                    "explorationItemAgeCutOff": "20"
                }
            }
        )
    elif trainingMode == "UPDATE" and campaign_arn is not None:
        create_campaign_response = personalize.update_campaign(
            campaignArn = campaign_arn,
            solutionVersionArn = solution_version_arn,
            minProvisionedTPS = 1,
            campaignConfig = {
                "itemExplorationConfig": {
                    "explorationWeight": "0.5",   ### Exploration 비율을 조정해 볼 수 있는 값
                    "explorationItemAgeCutOff": "20"
                }
            }
        )

    campaign_arn = create_campaign_response['campaignArn']
    print(json.dumps(create_campaign_response, indent=2))
except Exception as e:
    print(e)


{
  "campaignArn": "arn:aws:personalize:us-east-1:426989617652:campaign/personalize-sample-camp-20231012",
  "ResponseMetadata": {
    "RequestId": "2df665e0-6cac-4070-8032-a8999bf10ad8",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 09:59:48 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "102",
      "connection": "keep-alive",
      "x-amzn-requestid": "2df665e0-6cac-4070-8032-a8999bf10ad8"
    },
    "RetryAttempts": 0
  }
}


##### Wait for Campaign to Have ACTIVE Status

This should take about 10 minutes.

In [86]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_campaign_response = personalize.describe_campaign(
        campaignArn = campaign_arn
    )
    status = describe_campaign_response["campaign"]["status"]
    print("Campaign: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: CREATE IN_PROGRESS
Campaign: ACTIVE


## 7. Get Sample Recommendations



In [87]:
# Getting a random user:
USER_ID, ITEM_ID, _ ,_ ,_ ,_ = interaction_pd_data.sample().values[0]
print("USER: {}".format(USER_ID))

USER: 94


### 7-1. Call GetRecommendations

Using the user that you obtained above, the lines below will get recommendations for you and return the list of items that are recommended.


In [88]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list_first = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list_first:
    title = item['itemId']
    score = item['score']

    recommendation_list.append([title, score])
    
recommendations_df = pd.DataFrame(recommendation_list, columns = ['OriginalRecs-items','OriginalRecs-score'])
recommendations_df[:10]

Recommendations for user:  94


,OriginalRecs-items,OriginalRecs-score
0,1323,0.002645
1,1138,0.002506
2,1206,0.002430
3,1043,0.002112
4,1808,0.002099
5,1195,0.002075
6,1103,0.002075
7,1457,0.002058
8,1838,0.002046
9,1741,0.002038


### 7-3. Call GetRecommendations with Filter (exclude 'checked' and 'purchased')

In [89]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID),
    filterArn=res_filter['filterArn']
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = item['itemId']
    score = item['score']
    recommendation_list.append([title, score])
    

new_rec_DF = pd.DataFrame(recommendation_list, columns = ['Filtered-items','Filtered-score'])
try:
    recommendations_df.drop(['Filtered-items','Filtered-score'], axis=1, inplace=True)
except:
    pass

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df[:10]

Recommendations for user:  94


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score
0,1323,0.002645,1323,0.003234
1,1138,0.002506,1138,0.003064
2,1206,0.002430,1206,0.002971
3,1043,0.002112,1195,0.002537
4,1808,0.002099,1103,0.002537
5,1195,0.002075,1457,0.002517
6,1103,0.002075,1838,0.002501
7,1457,0.002058,1741,0.002492
8,1838,0.002046,1307,0.002484
9,1741,0.002038,1848,0.002477


In [90]:
filter_items_list = []
for inter_data in interaction_data:
    if inter_data[0] == USER_ID:
        filter_items_list.append(inter_data[1])
print(f"filter_items_list : {filter_items_list}")
for rec_list in recommendations_df['Filtered-items']:
    if rec_list in filter_items_list:
        print(f"rec_list : {rec_list}")

filter_items_list : [1303, 1294, 1528, 1087, 1423, 1703, 1215, 1461, 1817, 1811, 1994, 1692, 1108, 1008, 1359, 1747, 1515, 1043, 1378, 1328, 1888, 1943, 1499, 1461, 1711, 1559, 1475, 1944, 1994, 1356, 1444, 1316, 1126, 1470, 1873, 1243, 1894, 1332, 1786, 1707, 1916, 1763, 1135, 1962, 1608, 1661, 1103, 1692, 1586, 1822, 1965, 1905, 1801, 1310, 1531, 1690, 1493, 1303, 1181, 1743, 1031, 1335, 1963, 1822, 1391, 1721, 1791, 1857, 1350, 1856, 1967, 1467, 1039, 1205, 1274, 1047, 1416, 1966, 1381, 1094, 1896, 1567, 1030, 1584, 1798, 1245, 1852, 1032, 1056, 1867, 1889, 1674, 1344, 1174, 1427, 1828, 1372, 1795, 1078, 1168, 1937, 1110, 1663, 1313, 1813, 1996, 1085, 1919, 1643, 1444, 1325, 1596, 1895, 1992, 1880, 1467, 1012, 1176, 1710, 1242, 1836, 1515, 1740, 1115, 1792, 1678, 1252, 1195, 1258, 1561, 1561, 1696, 1127, 1671, 1810, 1919, 1327, 1006, 1384, 1432, 1735, 1442, 1852, 1425, 1134, 1719, 1381, 1161, 1119, 1377, 1608, 1235, 1414, 1747, 1231, 1036, 1969, 1994, 1037, 1108, 1619, 1746, 1492, 1

### 7-4. Creating an Event Tracker

Before your recommendation system can respond to real time events you will need an event tracker, the code below will generate one and can be used going forward with this lab. Feel free to name it something more clever.

In [91]:
# Imports
import boto3
import json
import numpy as np
import pandas as pd
import time
import uuid

In [92]:
# Setup and Config
# Recommendations from Event data
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's Event Streaming
personalize_events = boto3.client(service_name='personalize-events')

In [93]:
try:
    response = personalize.create_event_tracker(
        name=f'SampleTracker-{date}',
        datasetGroupArn=dataset_group_arn
    )
    event_tracker_arn = response['eventTrackerArn']
    TRACKING_ID = response['trackingId']
    print(f"event_tracker_arn : {event_tracker_arn} ,\nTRACKING_ID : {TRACKING_ID}")
except Exception as e:
    response = personalize.describe_event_tracker(
        eventTrackerArn=event_tracker_arn
    )
    event_tracker_arn=response['eventTracker']['eventTrackerArn']
    TRACKING_ID = response['eventTracker']['trackingId']
    print(f"event_tracker_arn : {event_tracker_arn} ,\nTRACKING_ID : {TRACKING_ID}")


event_tracker_arn : arn:aws:personalize:us-east-1:426989617652:event-tracker/f8fd1f35 ,
TRACKING_ID : 785619bb-bea7-4edd-bfa3-edf0413ddc37


#### 7-4-1. Put-Events with Filter

In [94]:
USER_ID = str(USER_ID)
ITEM_ID = recommendations_df['Filtered-items'][0]
print(f"USER_ID : {USER_ID}, ITEM_ID : {ITEM_ID}")

USER_ID : 94, ITEM_ID : 1323


In [96]:
session_dict = {}

# Configure Session
try:
    session_ID = session_dict[USER_ID]
except:
    session_dict[USER_ID] = str(uuid.uuid1())
    session_ID = session_dict[USER_ID]
    
print(f"session_ID : {session_ID}")
print(f"USER_ID : {USER_ID}")
print(f"ITEM_ID : {ITEM_ID}")

# Configure Properties:
event = {
    "itemId": ITEM_ID,
    "itemtype": "AAA"
}
event_json = json.dumps(event)

# Make Call
personalize_events.put_events(
trackingId = TRACKING_ID,
userId= USER_ID,
sessionId = session_ID,
eventList = [{
    'sentAt': int(time.time()),
    'recommendationId' : get_recommendations_response['recommendationId'],
    'eventType': 'checked',
    'properties': event_json
    }]
)

session_ID : 014fab26-91a3-11ed-aa9e-1688a5409149
USER_ID : 94
ITEM_ID : 1323


{'ResponseMetadata': {'RequestId': '0cacc972-0b0a-4bc9-8e5b-78e5e1df21d1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 11 Jan 2023 11:28:00 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0cacc972-0b0a-4bc9-8e5b-78e5e1df21d1'},
  'RetryAttempts': 0}}

In [100]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID),
    filterArn=res_filter['filterArn'] ### Filters
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = item['itemId']
    score = item['score']
    recommendation_list.append([title, score])
    

new_rec_DF = pd.DataFrame(recommendation_list, columns = ['Put-event-items','Put-event'])
try:
    recommendations_df.drop(['Put-event-items','Put-event'], axis=1, inplace=True)
except:
    pass

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df[:10]

Recommendations for user:  94


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score,Put-event-items,Put-event
0,1323,0.002645,1323,0.003234,1703,0.003096
1,1138,0.002506,1138,0.003064,1576,0.002792
2,1206,0.002430,1206,0.002971,1848,0.002718
3,1043,0.002112,1195,0.002537,1838,0.002700
4,1808,0.002099,1103,0.002537,1481,0.002677
5,1195,0.002075,1457,0.002517,1524,0.002642
6,1103,0.002075,1838,0.002501,1307,0.002633
7,1457,0.002058,1741,0.002492,1741,0.002587
8,1838,0.002046,1307,0.002484,1772,0.002581
9,1741,0.002038,1848,0.002477,1390,0.002458


### 7-5. Call GetRecommendations with Filter and Context

In [101]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID),
    filterArn=res_filter['filterArn'],
    context={
          'ITEM_TYPE': 'AAA'
      },
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = item['itemId']
    score = item['score']
    recommendation_list.append([title, score])
    

new_rec_DF = pd.DataFrame(recommendation_list, columns = ['Context-items','Context-score'])

try:
    recommendations_df.drop(['Context-items','Context-score'], axis=1, inplace=True)
except:
    pass

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df[:10]

Recommendations for user:  94


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score,Put-event-items,Put-event,Context-items,Context-score
0,1323,0.002645,1323,0.003234,1703,0.003096,1884,0.004258
1,1138,0.002506,1138,0.003064,1576,0.002792,1741,0.003902
2,1206,0.002430,1206,0.002971,1848,0.002718,1121,0.003767
3,1043,0.002112,1195,0.002537,1838,0.002700,1533,0.003749
4,1808,0.002099,1103,0.002537,1481,0.002677,1838,0.003578
5,1195,0.002075,1457,0.002517,1524,0.002642,1983,0.003534
6,1103,0.002075,1838,0.002501,1307,0.002633,1321,0.003460
7,1457,0.002058,1741,0.002492,1741,0.002587,1257,0.003387
8,1838,0.002046,1307,0.002484,1772,0.002581,1994,0.003228
9,1741,0.002038,1848,0.002477,1390,0.002458,1088,0.003220


### 7-6. Call GetRecommendations with Impression

In [102]:
USER_ID = str(USER_ID)
ITEM_ID = recommendations_df['Context-items'][0]
print(f"USER_ID : {USER_ID}, ITEM_ID : {ITEM_ID}")

USER_ID : 94, ITEM_ID : 1884


In [103]:
# Configure Session
try:
    session_ID = session_dict[USER_ID]
except:
    session_dict[USER_ID] = str(uuid.uuid1())
    session_ID = session_dict[USER_ID]
    
    
impression_list = [item[0] for item in recommendation_list[:5]]

print(f"session_ID : {session_ID}")
print(f"ITEM_ID : {ITEM_ID}")
print(f"impression_list : {impression_list}")

# Configure Properties:
event = {
    "itemId": ITEM_ID,
    "itemtype": "BBB"
}
event_json = json.dumps(event)

# Make Call
personalize_events.put_events(
trackingId = TRACKING_ID,
userId= USER_ID,
sessionId = session_ID,
eventList = [{
    'sentAt': int(time.time()),
    'recommendationId' : get_recommendations_response['recommendationId'],
    'impression': impression_list, ## Explicit impressions
    'eventType': 'purchased',
    'properties': event_json
    }]
)

session_ID : 014fab26-91a3-11ed-aa9e-1688a5409149
ITEM_ID : 1884
impression_list : ['1884', '1741', '1121', '1533', '1838']


{'ResponseMetadata': {'RequestId': '68654527-e0ed-47c8-b980-d571a2a23cc2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 11 Jan 2023 11:29:06 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '68654527-e0ed-47c8-b980-d571a2a23cc2'},
  'RetryAttempts': 0}}

In [104]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID),
    filterArn=res_filter['filterArn'],
    context={
          'ITEM_TYPE': 'AAA'
      },
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list = get_recommendations_response['itemList']

recommendation_list = []

for item in item_list:
    title = item['itemId']
    score = item['score']
    recommendation_list.append([title, score])
    

new_rec_DF = pd.DataFrame(recommendation_list, columns = ['Impression-items','Impression-score'])

try:
    recommendations_df.drop(['Impression-items','Impression-score'], axis=1, inplace=True)
except:
    pass

print(f"impression_list : {impression_list}")
recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  94
impression_list : ['1884', '1741', '1121', '1533', '1838']


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score,Put-event-items,Put-event,Context-items,Context-score,Impression-items,Impression-score
0,1323,0.002645,1323,0.003234,1703,0.003096,1884,0.004258,1196,0.005941
1,1138,0.002506,1138,0.003064,1576,0.002792,1741,0.003902,1983,0.004811
2,1206,0.002430,1206,0.002971,1848,0.002718,1121,0.003767,1733,0.004625
3,1043,0.002112,1195,0.002537,1838,0.002700,1533,0.003749,1731,0.004607
4,1808,0.002099,1103,0.002537,1481,0.002677,1838,0.003578,1121,0.004565
5,1195,0.002075,1457,0.002517,1524,0.002642,1983,0.003534,1874,0.004370
6,1103,0.002075,1838,0.002501,1307,0.002633,1321,0.003460,1847,0.004261
7,1457,0.002058,1741,0.002492,1741,0.002587,1257,0.003387,1449,0.004151
8,1838,0.002046,1307,0.002484,1772,0.002581,1994,0.003228,1897,0.004144
9,1741,0.002038,1848,0.002477,1390,0.002458,1088,0.003220,1342,0.004122


### 7-7. Call GetRecommendations after doing put_items

In [105]:
TIMESTAMP_10 = time.mktime((datetime.now()-timedelta(days=10)).timetuple())
TIMESTAMP_10 = int(TIMESTAMP_10)

TIMESTAMP_30 = time.mktime((datetime.now()-timedelta(days=30)).timetuple())
TIMESTAMP_30 = int(TIMESTAMP_30)
print(f"TIMESTAMP_10 : {TIMESTAMP_10}, TIMESTAMP_30 : {TIMESTAMP_30}")

TIMESTAMP_10 : 1672572725, TIMESTAMP_30 : 1670844725


In [106]:
new_ITEM_ID_10 = item_data[-1][0] +700000
new_ITEM_ID_30 = item_data[-1][0] +800000
print(f"new_ITEM_ID_10 : {new_ITEM_ID_10} , new_ITEM_ID_30 : {new_ITEM_ID_30}")

new_ITEM_ID_10 : 701999 , new_ITEM_ID_30 : 801999


In [118]:
items_10 = []
items_30 = []
for i in range(100):    
    new_ITEM_ID_30 = item_data[-1][0] + 800000 + i
    # Configure Properties:
    item_meta_30 = {
        'ITEM_META1': 'A_ITEM_1',
        'ITEM_META2':'B_ITEM_1',
        'CREATION_TIMESTAMP':TIMESTAMP_30
    }
    item_meta_json_30 = json.dumps(item_meta_30)
    
    items_30.append(
        {
            'itemId': str(new_ITEM_ID_30),
            'properties': item_meta_json_30
        }
    )
    if i%10 == 0:
        res_30 = personalize_events.put_items(
            datasetArn=items_dataset_arn,
            items=items_30
        )
        items_30 = []

    new_ITEM_ID_10 = item_data[-1][0] + 700000 + i
    # Configure Properties:
    item_meta_10 = {
        'ITEM_META1': 'A_ITEM_1',
        'ITEM_META2':'B_ITEM_1',
        'CREATION_TIMESTAMP':TIMESTAMP_10
    }
    item_meta_json_10 = json.dumps(item_meta_10)
    
    items_10.append(
        {
            'itemId': str(new_ITEM_ID_10),
            'properties': item_meta_json_10
        }
    )
    if i%10 == 0:
        res_10 = personalize_events.put_items(
            datasetArn=items_dataset_arn,
            items=items_10
        )
        items_10 = []
res_10, res_30

({'ResponseMetadata': {'RequestId': '26449b74-2631-464d-abb3-09b7e6ef77c3',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Wed, 11 Jan 2023 11:33:40 GMT',
    'content-type': 'application/json',
    'content-length': '0',
    'connection': 'keep-alive',
    'x-amzn-requestid': '26449b74-2631-464d-abb3-09b7e6ef77c3'},
   'RetryAttempts': 0}},
 {'ResponseMetadata': {'RequestId': '7d6174d3-64c0-4fbe-8759-96ab908bc0c2',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'date': 'Wed, 11 Jan 2023 11:33:40 GMT',
    'content-type': 'application/json',
    'content-length': '0',
    'connection': 'keep-alive',
    'x-amzn-requestid': '7d6174d3-64c0-4fbe-8759-96ab908bc0c2'},
   'RetryAttempts': 0}})

#### 7-7-1. Updating Solution

In [129]:
trainingMode='UPDATE'

In [131]:
try:
    create_solution_version_response = personalize.create_solution_version(
        solutionArn = solution_arn,
        trainingMode=trainingMode 
    )

    solution_version_arn = create_solution_version_response['solutionVersionArn']
    print(json.dumps(create_solution_version_response, indent=2))
except Exception as e:
    print(e)

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:426989617652:solution/personalize-sample-user-personalization-20231012/cbec35a4",
  "ResponseMetadata": {
    "RequestId": "21ef7b79-c5f3-4c77-9558-4a3774d02d97",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Wed, 11 Jan 2023 11:43:25 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "134",
      "connection": "keep-alive",
      "x-amzn-requestid": "21ef7b79-c5f3-4c77-9558-4a3774d02d97"
    },
    "RetryAttempts": 0
  }
}


In [132]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE PENDING
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: ACTIVE


In [134]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(USER_ID)
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", USER_ID)

item_list_first = get_recommendations_response['itemList']

recommendation_list_new_item = []

for item in item_list_first:
    title = item['itemId']
    score = item['score']
    recommendation_list_new_item.append([title, score])
    
new_rec_DF = pd.DataFrame(recommendation_list_new_item, columns = ['new-put-items','new-put-items-score'])

try:
    recommendations_df.drop(['new-put-items','new-put-items-score'], axis=1, inplace=True)
except:
    pass

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  94


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score,Put-event-items,Put-event,Context-items,Context-score,Impression-items,Impression-score,new-put-users,new-put-users-score,new-put-items,new-put-items-score
0,1323,0.002645,1323,0.003234,1703,0.003096,1884,0.004258,1196,0.005941,1206,0.002226,1838,0.002476
1,1138,0.002506,1138,0.003064,1576,0.002792,1741,0.003902,1983,0.004811,1037,0.002195,1273,0.002396
2,1206,0.002430,1206,0.002971,1848,0.002718,1121,0.003767,1733,0.004625,1124,0.002092,1647,0.002315
3,1043,0.002112,1195,0.002537,1838,0.002700,1533,0.003749,1731,0.004607,1219,0.002081,1755,0.002214
4,1808,0.002099,1103,0.002537,1481,0.002677,1838,0.003578,1121,0.004565,1323,0.002047,1524,0.002187
5,1195,0.002075,1457,0.002517,1524,0.002642,1983,0.003534,1874,0.004370,1735,0.001984,1242,0.002152
6,1103,0.002075,1838,0.002501,1307,0.002633,1321,0.003460,1847,0.004261,1105,0.001975,1460,0.002094
7,1457,0.002058,1741,0.002492,1741,0.002587,1257,0.003387,1449,0.004151,1808,0.001916,1481,0.002093
8,1838,0.002046,1307,0.002484,1772,0.002581,1994,0.003228,1897,0.004144,1457,0.001828,1023,0.002064
9,1741,0.002038,1848,0.002477,1390,0.002458,1088,0.003220,1342,0.004122,1481,0.001813,1949,0.001968


### 7-8. Call GetRecommendations after doing put_users

In [151]:
USER_ID

'94'

In [156]:
new_USER_ID = user_data[-1][0] + 100

print(f"USER_ID : {new_USER_ID}")

# Configure Properties:
user_meta = {
    'USER_META1': str(user_data[int(USER_ID)-1][1]),
    'USER_META2': user_data[int(USER_ID)-1][2],
    'CREATION_TIMESTAMP':int(time.time())
}
user_meta_json = json.dumps(user_meta)

personalize_events.put_users(
    datasetArn=users_dataset_arn,
    users=[
        {
            'userId': str(new_USER_ID),
            'properties': user_meta_json
        },
    ]
)

USER_ID : 200


{'ResponseMetadata': {'RequestId': '638c37af-3d0e-40d6-9a2c-770cca3cab31',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 11 Jan 2023 12:30:54 GMT',
   'content-type': 'application/json',
   'content-length': '0',
   'connection': 'keep-alive',
   'x-amzn-requestid': '638c37af-3d0e-40d6-9a2c-770cca3cab31'},
  'RetryAttempts': 0}}

In [157]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = campaign_arn,
    userId = str(new_USER_ID),
)
# Update DF rendering
pd.set_option('display.max_rows', 30)

print("Recommendations for user: ", new_USER_ID)

item_list_first = get_recommendations_response['itemList']

recommendation_list_new_user = []

for item in item_list_first:
    title = item['itemId']
    score = item['score']
    recommendation_list_new_user.append([title, score])
    
new_rec_DF = pd.DataFrame(recommendation_list_new_user, columns = ['new-put-users','new-put-users-score'])

try:
    recommendations_df.drop(['new-put-users','new-put-users-score'], axis=1, inplace=True)
except:
    pass

recommendations_df = recommendations_df.join(new_rec_DF)
recommendations_df

Recommendations for user:  200


,OriginalRecs-items,OriginalRecs-score,Filtered-items,Filtered-score,Put-event-items,Put-event,Context-items,Context-score,Impression-items,Impression-score,new-put-items,new-put-items-score,new-put-users,new-put-users-score
0,1323,0.002645,1323,0.003234,1703,0.003096,1884,0.004258,1196,0.005941,1838,0.002476,1206,0.002226
1,1138,0.002506,1138,0.003064,1576,0.002792,1741,0.003902,1983,0.004811,1273,0.002396,1037,0.002195
2,1206,0.002430,1206,0.002971,1848,0.002718,1121,0.003767,1733,0.004625,1647,0.002315,1124,0.002092
3,1043,0.002112,1195,0.002537,1838,0.002700,1533,0.003749,1731,0.004607,1755,0.002214,1219,0.002081
4,1808,0.002099,1103,0.002537,1481,0.002677,1838,0.003578,1121,0.004565,1524,0.002187,1323,0.002047
5,1195,0.002075,1457,0.002517,1524,0.002642,1983,0.003534,1874,0.004370,1242,0.002152,1735,0.001984
6,1103,0.002075,1838,0.002501,1307,0.002633,1321,0.003460,1847,0.004261,1460,0.002094,1105,0.001975
7,1457,0.002058,1741,0.002492,1741,0.002587,1257,0.003387,1449,0.004151,1481,0.002093,1808,0.001916
8,1838,0.002046,1307,0.002484,1772,0.002581,1994,0.003228,1897,0.004144,1023,0.002064,1457,0.001828
9,1741,0.002038,1848,0.002477,1390,0.002458,1088,0.003220,1342,0.004122,1949,0.001968,1481,0.001813


In [138]:
for val in list(recommendations_df['OriginalRecs-items']):
    if val in list(recommendations_df['new-put-users']):
        print(val)

1323
1138
1206
1043
1808
1400
1387
1521
1037


## Review

Using the codes above you have successfully trained a deep learning model to generate recommendations based on prior user behavior. Think about other types of problems where this data is available and what it might look like to build a system like this to offer those recommendations.


- [Amazon Personalize Immersion Day](https://catalog.us-east-1.prod.workshops.aws/workshops/c5a0c80f-1a42-442c-b2c0-956b38d4dc48/en-US) 
- [amazon-personalize-samples](https://github.com/aws-samples/amazon-personalize-samples)
- [Amazon Personalize 기반으로 실시간 추천 사이트 만들기](https://catalog.us-east-1.prod.workshops.aws/workshops/ed82a5d4-6630-41f0-a6a1-9345898fa6ec/ko-KR)

## Delete Personalize

In [ ]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
region_name = boto3.Session().region_name

In [ ]:
delete_date='20221023'

In [ ]:
for list_event_tracker in personalize.list_event_trackers()['eventTrackers']:
    del_event_tracker = list_event_tracker['name']
    if del_event_tracker == f'SampleTracker-{delete_date}':
        eventTrackerArn= list_event_tracker['eventTrackerArn']
        personalize.delete_event_tracker(eventTrackerArn=eventTrackerArn)
        print(f'del_event_tracker : {del_event_tracker}')
        print(f'eventTrackerArn : {eventTrackerArn}')

In [ ]:
filterArn=f'arn:aws:personalize:{region_name}:{account_id}:filter/sample-filter-not-in-check-and-purchased-{delete_date}'
campaignArn=f'arn:aws:personalize:{region_name}:{account_id}:campaign/personalize-sample-camp-{delete_date}'
try:
    personalize.delete_filter(filterArn=filterArn)
    personalize.delete_campaign(campaignArn=campaignArn)

except Exception as e:
    print(e)
    

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    try:
        describe_filter_response = personalize.describe_filter(
            filterArn = filterArn
        )
        status = describe_filter_response["filter"]["status"]
        print("Filter: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(5)
while time.time() < max_time:
    try:
        describe_campaign_response = personalize.describe_campaign(
            campaignArn = campaignArn
        )
        status = describe_campaign_response["campaign"]["status"]
        print("Campaign: {}".format(status))
        
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(5)

In [ ]:
solutionArn=f'arn:aws:personalize:{region_name}:{account_id}:solution/personalize-sample-user-personalization-{delete_date}'
    
try:
    personalize.delete_solution(solutionArn=solutionArn)

except Exception as e:
    print(e)
    

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    try:
        describe_solution_response = personalize.describe_solution(
            solutionArn = solutionArn
        )
        status = describe_solution_response["solution"]["status"]
        print("Solution: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(10)

In [ ]:
datasetArn_base=f'arn:aws:personalize:{region_name}:{account_id}:dataset/dataset-samples-{delete_date}'
    
try:
    personalize.delete_dataset(datasetArn=f'{datasetArn_base}/INTERACTIONS')
    personalize.delete_dataset(datasetArn=f'{datasetArn_base}/USERS')
    personalize.delete_dataset(datasetArn=f'{datasetArn_base}/ITEMS')

except Exception as e:
    print(e)
    

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    try:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = f'{datasetArn_base}/INTERACTIONS'
        )
        status = describe_dataset_response["dataset"]["status"]
        print("INTERACTIONS: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(5)

while time.time() < max_time:
    try:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = f'{datasetArn_base}/USERS'
        )
        status = describe_dataset_response["dataset"]["status"]
        print("USERS: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(5)
    
while time.time() < max_time:
    try:
        describe_dataset_response = personalize.describe_dataset(
            datasetArn = f'{datasetArn_base}/ITEMS'
        )
        status = describe_dataset_response["dataset"]["status"]
        print("ITEMS: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(5)

In [ ]:
datasetGroupArn=f'arn:aws:personalize:{region_name}:{account_id}:dataset-group/dataset-samples-{delete_date}'
    
try:
    personalize.delete_dataset_group(datasetGroupArn=datasetGroupArn)

except Exception as e:
    print(e)
    

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    try:
        describe_dataset_group_response = personalize.describe_dataset_group(
            datasetGroupArn = datasetGroupArn
        )
        status = describe_dataset_group_response["datasetGroup"]["status"]
        print("datasetGroup: {}".format(status))
        
        if status == "ACTIVE" or status == "CREATE FAILED":
            break

    except Exception as e:
        print(e)
        break

    time.sleep(10)